In [4]:
import json
import numpy as np
import pandas as pd
import psycopg2

## Import and format the data

First, connect to the database.

In [5]:
with open("config.json") as f:
    conf = json.load(f)

In [6]:
conn_str = "host={} dbname={} user={} password={}".format(conf["host"], conf["database"], conf["user"], conf["password"])

In [7]:
conn = psycopg2.connect(conn_str)

Query the database to return the shit we want.

In [ ]:
broadband_16 = pd.read_sql("""SELECT block_pops.state,
       block_pops.county,
       block_pops.population,
       block_pops.sq_miles,
       block_pops.pop_per_sq_mile,
       block_pops.full_fips,
       deployment_2016.blockcode
FROM block_pops
LEFT JOIN deployment_2016
WHERE deployment_2016.consumer = '1'
  AND deployment_2016.maxaddown >= 25
  AND deployment_2016.maxadup >= 3
  AND (deployment_2016.techcode = '10'
       OR deployment_2016.techcode = '11'
       OR deployment_2016.techcode= '12'
       OR deployment_2016.techcode = '20'
       OR deployment_2016.techcode = '30'
       OR deployment_2016.techcode = '40'
       OR deployment_2016.techcode = '41'
       OR deployment_2016.techcode = '42'
       OR deployment_2016.techcode = '43'
       OR deployment_2016.techcode = '50')
GROUP BY block_pops.state,
       block_pops.county,
       block_pops.population,
       block_pops.sq_miles,
       block_pops.pop_per_sq_mile,
       block_pops.full_fips,
       deployment_2016.blockcode;""",
                           con=conn)